In [ ]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import re
from datetime import datetime
import pandas as pd

# Configurando as opções do Chrome para executar headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ativar o modo headless
service = ChromeService(ChromeDriverManager().install())




In [ ]:


# driver = webdriver.Chrome(service=service)
driver = webdriver.Chrome(service=service, options=chrome_options)


url = "https://g1.globo.com/tudo-sobre/petrobras/"

driver.get(url)

time.sleep(5)

cookie = driver.find_element(
    By.XPATH, '//*[@id="cookie-banner-lgpd"]/div/div[2]/button'
)
cookie.click()


regex_data = r"/(\d{4}/\d{2}/\d{2})/"
formato = "%Y/%m/%d"


def busca_link(dados):
    link = (
        dados.find_element(By.CLASS_NAME, "feed-post-body")
        .find_element(By.TAG_NAME, "a")
        .get_attribute("href")
    )
    return link


def transforma_data(dados):
    match = re.search(regex_data, dados)
    if match:
        data_str = match.group(1)
        data = datetime.strptime(data_str, formato)
    else:
        data = None
    return data


def busca_link(dados):
    link = (
        dados.find_element(By.CLASS_NAME, "feed-post-body")
        .find_element(By.TAG_NAME, "a")
        .get_attribute("href")
    )
    return link


def busca_botao_veja_mais():
    veja_mais = driver.find_element(
        By.XPATH,
        "//*[@id='feed-placeholder']/div/div/div[3]/a[contains(text(), 'Veja mais')]",
    )
    return veja_mais


def busca_conteudo():
    conteudo = driver.find_elements(
        By.XPATH,
        '//*[contains(concat( " ", @class, " " ), concat( " ", "type-materia", " " ))]',
    )
    return conteudo


def obtem_titulo(dados):
    titulo = (
        dados.find_element(By.CLASS_NAME, "feed-post-body")
        .find_element(By.TAG_NAME, "a")
        .text
    )
    return titulo


limit = True
while limit:
    veja_mais = busca_botao_veja_mais()
    veja_mais.click()
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(5)
    conteudo = busca_conteudo()
    link = (
        busca_link(conteudo[-1])
        if busca_link(conteudo[-1])
        else busca_link(conteudo[-2])
    )
    data = transforma_data(link)
    data_limite = datetime(2022, 1, 1)
    print(data)
    if data is not None and data < data_limite:
        limit = False
        break




In [ ]:



dados = []

for i, titulo in enumerate(conteudo):
    link = busca_link(conteudo[i])
    titulo_texto = obtem_titulo(conteudo[i])
    data = transforma_data(link)

    dados.append((data, titulo_texto, link))

df = pd.DataFrame(dados, columns=["Data", "Título", "Link"])



df.to_excel("dados.xlsx", index=False)

